## Crossmatch API 

### Issue [lsdb#946](https://github.com/astronomy-commons/lsdb/issues/946)

The current crossmatch API accepts:
- Algorithm class type (`algorithm`)
- Algorithm arguments (`**kwargs`)

```python
def crossmatch(
    self,
    other: Catalog,
    suffixes: tuple[str, str] | None = None,
    algorithm: (
        type[AbstractCrossmatchAlgorithm] | BuiltInCrossmatchAlgorithm
    ) = BuiltInCrossmatchAlgorithm.KD_TREE,
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffix_method: str | None = None,
    log_changes: bool = True,
    **kwargs,
) -> Catalog:
```

We want to make sure the crossmatch arguments (`**kwargs`) are explicitly defined and recognized by IDEs.

### Option A

```python
def crossmatch(
    self,
    other: Catalog,
    # Default KDTree kwargs ----
    n_neighbors: int | None = None,
    radius_arcsec: float | None = None,
    min_radius_arcsec: float | None = None,
    # Custom algorithm ---------
    algorithm: CrossmatchAlgorithm | None = None,
    # --------------------------
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffixes: tuple[str, str] | None = None,
    suffix_method: str | None = None,
    log_changes: bool = True,
) -> Catalog:
```

__Pros:__
- We are able to have a single method for crossmatching.
- The user does not need to instantiate a `KDTreeCrossmatchAlgorithm` object, they can use the explicitly defined kwargs:

    ```python
    # Algorithm is not specified, so KDTree crossmatch is used
    ztf.crossmatch(gaia, n_neighbors=3, radius_arcsec=5)
    ```

__Cons:__
- If a custom algorithm defines attributes with the same name as the default kwargs, we need to make sure they do not conflict: 

    ```python
    # This should raise an error since we have conflicting kwargs
    class MyAlgorithm(CrossmatchAlgorithm):
        min_radius_arcsec=0.0
    ztf.crossmatch(gaia, algorithm=MyAlgorithm(), min_radius_arcsec=2)
    ```
- The default values for `n_neighbors`, `radius_arcsec` and `min_radius_arcsec` need to be None for us to check if the user set those or not. We would define the default values in the docstring, but it's not as explicit.

__Also:__

To initialize other "built-in" algorithms, the user would import the method and pass it as a custom algorithm:

```python
from lsdb.core.crossmatch.builtin_algorithms import OtherCrossmatch
ztf.crossmatch(gaia, algorithm=OtherCrossmatch(...))

### Option B

Similar but we split the default crossmatch into a separate method:

```python
# Default crossmatch (KDTree)
def crossmatch(
    self,
    other: Catalog,
    n_neighbors: int = 1,
    radius_arcsec: float = 1,
    min_radius_arcsec: float = 0.0,
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffixes: tuple[str, str] | None = None,
    suffix_method: str | None = None,
    log_changes: bool = True,
) -> Catalog:

# Every other crossmatch
def custom_crossmatch(
    self,
    other: Catalog,
    algorithm: CrossmatchAlgorithm | None = None,
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffixes: tuple[str, str] | None = None,
    suffix_method: str | None = None,
    log_changes: bool = True,
) -> Catalog:
```

__Pros:__
- There's a clear separation of concerns - each method only has the behavior/arguments it needs.
- The default values for `n_neighbors`, `radius_arcsec` and `min_radius_arcsec` can be set directly in the signature:

    ```python
    # E.g. this would use the default n_neighbors=1
    ztf.crossmatch(gaia, radius_arcsec=5, min_radius_arcsec=2)
    ```

__Cons:__
- Since we also have `Catalog.crossmatch_nested` we would end up with 4 methods for crossmatching (instead of just `Catalog.crossmatch` and `Catalog.crossmatch_nested`). We could add the "nested crossmatch" behavior to these methods but it would probably be confusing to have the "suffixes" kwargs in the signature if they're not being used for nesting (and vice-versa):

    ```python
    def crossmatch(
        self,
        other: Catalog,
        n_neighbors: int = 1,
        radius_arcsec: float = 1,
        min_radius_arcsec: float = 0.0,
        output_catalog_name: str | None = None,
        require_right_margin: bool = False,
        suffixes: tuple[str, str] | None = None,
        suffix_method: str | None = None,
        log_changes: bool = True,
        nested_column_name: str | None = None, # For nesting
    ) -> Catalog:

    # The result is nested. suffixes, suffix_method, log_changes kwargs are ignored.
    ztf.crossmatch(ztf_sources, nested_column_name="sources")

    # The result is not nested. nested_column_name kwarg is ignored.
    ztf.crossmatch(ztf_sources, suffixes=["_ztf","_sources"])

    # Since nested_column_name is set, we can assume the result should be nested (and ignore suffixes).
    # But maybe we should just raise an error?
    ztf.crossmatch(ztf_sources, nested_column_name="sources", suffixes=["_ztf","_sources"])
    ```

__Also:__

To initialize other "built-in" algorithms, the user would import the method and pass it as a custom algorithm:
```python
from lsdb.core.crossmatch.builtin_algorithms import OtherCrossmatch
ztf.custom_crossmatch(gaia, algorithm=OtherCrossmatch(...))